<a href="https://colab.research.google.com/github/PedroFerrerRincon/Adjusted-Momentim-Trading-Algorythm/blob/main/Equity_Sector_Rotaion_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To do : low epochs 2000 first neuron

V2 Changes - Added "Earnings Growth Q-1", "Earnings Growth Q-2", "Financing Gap" - Added oversampling drwdowns - removed custom loss function - created six folder system - added excel summary - added window variable - validation from 2022

V3 Changes - Added hyperparameter combination

V4 Changes - Added Skewness, Kurtosis, Beta and Returns Plots

In [ ]:
import math
import numpy as np
import pandas as pd
import datetime
import random
import os
import shutil  # For copying the checkpoint file
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import zipfile

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

# For reproducibility
RANDOM_STATE = 42
def set_random_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

set_random_seeds(RANDOM_STATE)

# ---------------------------------
# 2) User Configuration
# ---------------------------------
FILE_NAME = 'AllDataTraining.csv'

# The equity price/index columns we want to predict 180-step fwd returns on:
SECTOR_PRICE_COLS = [
    "S&P GL 1200 Energy",
    "S&P GL 1200 Financials",
    "S&P GL 1200 Health Care",
    "S&P GL 1200 Materials",
    "S&P GL 1200 Industrials",
    "S&P GL 1200 Consumer Staples",
    "S&P GL 1200 Utilities",
    "S&P GL 1200 Consumer Discretionary",
    "S&P GL 1200 Communication Services",
    "S&P GL 1200 Information Technology"
]

WINDOW = 180

# Benchmark price/index column:
BENCHMARK_PRICE_COL = "SP500"

# Input features for the LSTM:
INPUT_COLS = [
    'CPI', 'Unemployment', 'GDP Nominal', 'GDP Real',
    'M2 Velocity', 'M2', 'Oil Prices USD', 'Credit Spreads',
    '1 Mo', '3 Mo', '6 Mo', '1 Yr', '2 Yr', '3 Yr',
    '5 Yr', '7 Yr', '10 Yr', '20 Yr', '30 Yr',
    'Baltic Dry Index', 'ISM',
    '1-3y t-60', '1-3y t-180', '4-7y t-60', '4-7y t-180',
    '7-10y t-60', '7-10y t-180',
    "SNP500 t-60", "SNP500 t-180", "SNP500 VOL30",
    "EURUSD", "EURUSD t-30","EURUSD t-60",
    'Drawdown Spread', '1-3yr Drawdown', '7-10yr Drawdown',
]

# Candidate lag sets for the LSTM:
LAGS_GRID = [
    [0, 5, 21, 64, 255],  # example lags
]

# Hyperparameters: now we will iterate over all elements in these lists
EPOCHS          = [100000]
BATCH_SIZE      = [1028]
DROPOUT         = [0]
LEARNING_RATE   = [0.0001]
REG_LAMBDA      = [0]
NUM_LSTM_LAYERS = [4]
FIRST_LAYER_SIZE= [1500]

# Train/Validation date ranges:
TRAIN1_START = pd.to_datetime("1994-12-30")
TRAIN1_END   = pd.to_datetime("2000-12-29")
VALIDATION_START = pd.to_datetime("2001-01-02")
VALIDATION_END   = pd.to_datetime("2010-12-31")
TRAIN2_START = pd.to_datetime("2011-01-03")
TRAIN2_END   = pd.to_datetime("2021-12-17")

# Rebalancing frequency (days)
REBALANCE_FREQUENCY_DAYS = 5

# Output directory
MAIN_OUTPUT_DIR = "grid_search_results_LSTM"
os.makedirs(MAIN_OUTPUT_DIR, exist_ok=True)

# Additional subfolder names (these will live within each combo directory)
SUBFOLDER_25pct_2001_2011       = "portfolio_25pct_2001_2011"
SUBFOLDER_cascading_2001_2011   = "portfolio_cascading_2001_2011"
SUBFOLDER_25pct_2023_2024       = "portfolio_25pct_2023_2024"
SUBFOLDER_cascading_2023_2024   = "portfolio_cascading_2023_2024"
SUBFOLDER_sectors_2001_2011     = "sector_predictions_2001_2011"
SUBFOLDER_sectors_2023_2024     = "sector_predictions_2023_2024"

# ---------------------------------
# 3) Data Loading and Functions
# ---------------------------------
def load_data(csv_path: str) -> pd.DataFrame:
    """Loads CSV with 'Date' column, sets Date as index (datetime)."""
    df = pd.read_csv(csv_path, parse_dates=['Date'], dayfirst=True, low_memory=False)
    df.set_index('Date', inplace=True)
    df.sort_index(inplace=True)
    df = df[df.index.weekday < 5]  # Keep only Mon-Fri if that was intended
    return df

def add_daily_return_columns(df: pd.DataFrame, price_cols: list) -> pd.DataFrame:
    """
    For each price/index column in price_cols, create col+"_daily_ret" = daily returns.
    """
    df = df.copy()
    for col in price_cols:
        daily_ret_col = col + "_daily_ret"
        df[daily_ret_col] = df[col].pct_change(fill_method=None)
    return df

def create_180d_forward_returns(df: pd.DataFrame, daily_return_cols: list, window=180) -> pd.DataFrame:
    """
    For each *daily return* column in daily_return_cols, compute the 180-day forward return.
    """
    df = df.copy()
    n = len(df)
    for col in daily_return_cols:
        fwd_name = col + "_fwd180"
        fwd_vals = [np.nan] * n
        for i in range(n - window):
            window_returns = df[col].iloc[i+1 : i+1+window]
            if window_returns.isnull().any():
                fwd_vals[i] = np.nan
            else:
                fwd_vals[i] = np.prod(1.0 + window_returns.values) - 1.0
        df[fwd_name] = fwd_vals
    return df

def split_three_periods_precomputed_data(X, y, dates,
                                        train1_start, train1_end,
                                        validation_start, validation_end,
                                        train2_start, train2_end):
    """
    Split (X, y) into train (train1 + train2) and validation sets.
    """
    dates = pd.to_datetime(dates)
    train1_idx = [i for i, d in enumerate(dates) if train1_start <= d <= train1_end]
    valid_idx  = [i for i, d in enumerate(dates) if validation_start <= d <= validation_end]
    train2_idx = [i for i, d in enumerate(dates) if train2_start <= d <= train2_end]

    if len(train1_idx) == 0 or len(train2_idx) == 0 or len(valid_idx) == 0:
        raise ValueError("One or more split periods are empty. Check your date ranges.")

    X_train = np.concatenate((X[train1_idx], X[train2_idx]), axis=0)
    y_train = np.concatenate((y[train1_idx], y[train2_idx]), axis=0)
    train_dates = [dates[i] for i in (train1_idx + train2_idx)]

    X_eval = X[valid_idx]
    y_eval = y[valid_idx]
    eval_dates = [dates[i] for i in valid_idx]

    return (X_train, y_train, train_dates), (X_eval, y_eval, eval_dates)

def prepare_lagged_data(df: pd.DataFrame, input_cols: list, label_cols: list, lags: list):
    """
    Create X, y, valid_dates for an LSTM with the given lags.
    X.shape = (samples, len(lags), len(input_cols))
    y.shape = (samples, len(label_cols))
    """
    df = df.sort_index()
    X_list, y_list, valid_dates = [], [], []
    all_dates = df.index.unique().sort_values()

    for i, current_date in enumerate(all_dates):
        lag_data = []
        missing_lag = False
        for lag in lags:
            lag_idx = i - lag
            if lag_idx < 0:
                missing_lag = True
                break
            lag_date = all_dates[lag_idx]
            if lag_date not in df.index:
                missing_lag = True
                break
            row = df.loc[lag_date, input_cols]
            if isinstance(row, pd.DataFrame):
                row = row.iloc[0]
            if row.isnull().any():
                missing_lag = True
                break
            lag_data.append(row.values.astype(float))
        if missing_lag or (current_date not in df.index):
            continue

        label_val = df.loc[current_date, label_cols]
        if isinstance(label_val, pd.DataFrame):
            label_val = label_val.iloc[0]
        if label_val.isnull().any():
            continue

        X_list.append(np.array(lag_data))
        y_list.append(label_val.values.astype(float))
        valid_dates.append(current_date)

    X_arr = np.array(X_list)
    y_arr = np.array(y_list)
    return X_arr, y_arr, valid_dates

# ---------------------------------
# 4) Model and Training
# ---------------------------------
def build_lstm_model(n_lags, n_features, n_outputs,
                     num_lstm_layers=2, first_layer_size=64,
                     dropout_rate=0.0, reg_lambda=0.0,
                     learning_rate=0.001):
    """
    Build a multi-layer LSTM with halving layer sizes, culminating in Dense(n_outputs).
    No custom loss; just 'mse'.
    """
    layer_sizes = [first_layer_size]
    for _ in range(1, num_lstm_layers):
        layer_sizes.append(max(1, layer_sizes[-1]//2))

    l2_reg = regularizers.l2(reg_lambda)
    model = Sequential()
    for i, units in enumerate(layer_sizes):
        return_seq = (i < num_lstm_layers - 1)
        if i == 0:
            model.add(
                LSTM(units,
                     return_sequences=return_seq,
                     input_shape=(n_lags, n_features),
                     kernel_regularizer=l2_reg)
            )
        else:
            model.add(
                LSTM(units,
                     return_sequences=return_seq,
                     kernel_regularizer=l2_reg)
            )
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))

    model.add(Dense(n_outputs, kernel_regularizer=l2_reg))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    return model

# ---------------------------------
# 4B) Custom Callback for Post-15000 Behavior
# ---------------------------------
class Post15000EarlyStopping(tf.keras.callbacks.Callback):
    """
    After epoch 15,000, if the current validation loss is more than 5% above
    the best (lowest) val_loss (which we set starting from epoch 15,000),
    we stop training. Otherwise, if val_loss sets a new low, we update
    best_val_loss and save the model.
    """

    def __init__(self, best_model_path="temp_best_model.keras"):
        super().__init__()
        self.best_val_loss = None
        self.best_model_path = best_model_path
        self.baseline_set = False   # Will become True at epoch == 15000

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        if val_loss is None:
            return

        current_epoch = epoch + 1
        # Ignore everything before epoch 15,000
        if current_epoch < 15000:
            return

        # At epoch 15,000, set the baseline (best_val_loss = current val_loss)
        if current_epoch == 15000 and not self.baseline_set:
            self.best_val_loss = val_loss
            self.baseline_set = True
            # Save model at epoch 15,000 (baseline checkpoint)
            self.model.save(self.best_model_path)
            return

        # Beyond epoch 15,000
        if not self.baseline_set:
            # If for some reason we missed it, we set baseline now
            self.best_val_loss = val_loss
            self.baseline_set = True
            self.model.save(self.best_model_path)
            return

        # If val_loss is < best_val_loss, update
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.model.save(self.best_model_path)
        else:
            # If val_loss > 1.05 * self.best_val_loss => STOP
            if val_loss > 1.05 * self.best_val_loss:
                print(
                    f"Early stopping at epoch {current_epoch} because val_loss "
                    f"({val_loss:.6f}) exceeded 105% of best_val_loss "
                    f"({self.best_val_loss:.6f})."
                )
                self.model.stop_training = True

# ---------------------------------
# 5) Backtesting + Heatmaps
# ---------------------------------
def backtest_top4_portfolio(dates_eval, y_pred, df_eval,
                            sector_daily_ret_cols, benchmark_daily_ret_col,
                            rebal_freq=5):
    """
    We do rebalancing every 'rebal_freq' days for a 25%-each approach:
      - On rebal day i, pick top-4 predicted from y_pred[i-1], each 25%.
    """
    df_backtest = df_eval.loc[dates_eval, sector_daily_ret_cols + [benchmark_daily_ret_col]].copy()
    df_backtest.sort_index(inplace=True)

    n = len(df_backtest)
    strategy_nav = np.ones(n)
    benchmark_nav = np.ones(n)
    num_sectors = len(sector_daily_ret_cols)
    top4_matrix = np.zeros((n, num_sectors), dtype=int)

    current_top4 = []
    for i in range(n):
        if i == 0 or (i % rebal_freq == 0):
            # On rebal day, pick top-4 from y_pred[i-1], but i-1 must be valid
            if i == 0:
                current_top4 = []
            else:
                pred_prev = y_pred[i - 1]
                top4 = np.argsort(pred_prev)[-4:]
                current_top4 = top4

        day_returns = df_backtest[sector_daily_ret_cols].iloc[i].values
        if len(current_top4) > 0:
            top4_matrix[i, current_top4] = 1
            strat_daily_return = np.mean(day_returns[current_top4])
        else:
            strat_daily_return = 0.0

        if i > 0:
            strategy_nav[i] = strategy_nav[i-1] * (1.0 + strat_daily_return)
        else:
            strategy_nav[i] = 1.0 * (1.0 + strat_daily_return)

        bm_ret = df_backtest[benchmark_daily_ret_col].iloc[i]
        if i > 0:
            benchmark_nav[i] = benchmark_nav[i-1] * (1.0 + bm_ret)
        else:
            benchmark_nav[i] = 1.0 * (1.0 + bm_ret)

    nav_df = pd.DataFrame({
        "Strategy_NAV": strategy_nav,
        "Benchmark_NAV": benchmark_nav
    }, index=df_backtest.index)
    return nav_df, top4_matrix

def backtest_top4_cascading_weights(dates_eval, y_pred, df_eval,
                                    sector_daily_ret_cols, benchmark_daily_ret_col,
                                    rebal_freq=5):
    """
    A backtest that applies weights [0.4, 0.3, 0.2, 0.1] to the top 4 predicted.
    """
    df_backtest = df_eval.loc[dates_eval, sector_daily_ret_cols + [benchmark_daily_ret_col]].copy()
    df_backtest.sort_index(inplace=True)

    n = len(df_backtest)
    strategy_nav = np.ones(n)
    benchmark_nav = np.ones(n)
    num_sectors = len(sector_daily_ret_cols)

    daily_weights = np.zeros((n, num_sectors), dtype=float)
    top_weights = [0.4, 0.3, 0.2, 0.1]
    current_weights = np.zeros(num_sectors, dtype=float)

    for i in range(n):
        if i == 0 or (i % rebal_freq == 0):
            # On rebal day, pick top-4 from y_pred[i-1]
            if i == 0:
                current_weights = np.zeros(num_sectors)
            else:
                pred_prev = y_pred[i - 1]
                sorted_idx = np.argsort(pred_prev)[::-1]  # descending
                top4 = sorted_idx[:4]
                w_new = np.zeros(num_sectors)
                for rank, sec_idx in enumerate(top4):
                    w_new[sec_idx] = top_weights[rank]
                current_weights = w_new

        daily_weights[i] = current_weights
        day_returns = df_backtest[sector_daily_ret_cols].iloc[i].values
        strat_daily_return = np.dot(current_weights, day_returns)

        if i > 0:
            strategy_nav[i] = strategy_nav[i - 1] * (1.0 + strat_daily_return)
        else:
            strategy_nav[i] = 1.0 * (1.0 + strat_daily_return)

        bm_ret = df_backtest[benchmark_daily_ret_col].iloc[i]
        if i > 0:
            benchmark_nav[i] = benchmark_nav[i - 1] * (1.0 + bm_ret)
        else:
            benchmark_nav[i] = 1.0 * (1.0 + bm_ret)

    nav_df = pd.DataFrame({
        "Strategy_NAV": strategy_nav,
        "Benchmark_NAV": benchmark_nav
    }, index=df_backtest.index)
    return nav_df, daily_weights

def compute_performance(nav_series: pd.Series):
    """
    Return dict of performance stats: total return, CAGR, vol, sharpe, max drawdown.
    (Skew/Kurt/Beta are added below, in the main script.)
    """
    s = nav_series.copy().dropna()
    if len(s) < 2:
        return {}
    total_return = s.iloc[-1] / s.iloc[0] - 1
    days = (s.index[-1] - s.index[0]).days
    years = days / 365.25 if days > 0 else 1e-9
    cagr = (1 + total_return)**(1/years) - 1 if years>0 else np.nan
    daily_ret = s.pct_change().dropna()
    vol = daily_ret.std() * np.sqrt(252) if len(daily_ret) > 1 else np.nan
    sharpe = 0
    if daily_ret.std() and daily_ret.std() != 0:
        sharpe = (daily_ret.mean() / daily_ret.std()) * np.sqrt(252)
    dd = (s / s.cummax()) - 1
    max_dd = dd.min()
    return {
        "Total_Return": total_return,
        "CAGR": cagr,
        "Volatility": vol,
        "Sharpe_Ratio": sharpe,
        "Max_Drawdown_pct": max_dd * 100
    }

def plot_drawdowns(nav_df, output_path):
    """
    Plot drawdown lines for Strategy and Benchmark.
    """
    plt.figure(figsize=(10,5))
    s_strat = nav_df["Strategy_NAV"]
    dd_strat = s_strat / s_strat.cummax() - 1

    s_bench = nav_df["Benchmark_NAV"]
    dd_bench = s_bench / s_bench.cummax() - 1

    plt.plot(dd_strat.index, dd_strat, label="Strategy")
    plt.plot(dd_bench.index, dd_bench, label="Benchmark")
    plt.title("Drawdowns")
    plt.legend()
    plt.grid(True)
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def plot_top4_heatmap(dates, top4_matrix, sector_labels, output_path):
    """
    Heatmap for the 25%-each approach (1=selected, 0=not).
    """
    fig, ax = plt.subplots(figsize=(12,6))
    cax = ax.imshow(top4_matrix.T, aspect='auto', interpolation='nearest', origin='lower')
    ax.set_yticks(range(len(sector_labels)))
    ax.set_yticklabels(sector_labels)
    step = max(1, len(dates)//10)
    ax.set_xticks(range(0, len(dates), step))
    xlabels = [dates[i].strftime('%Y-%m-%d') for i in range(0,len(dates), step)]
    ax.set_xticklabels(xlabels, rotation=45, ha='right')
    fig.colorbar(cax, ax=ax, label="1=In Top 4, 0=Not in Top 4")
    ax.set_title("Heatmap: 25%-each-of-top-4")
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

def plot_cascading_weight_heatmap(dates, weight_matrix, sector_labels, output_path):
    """
    Heatmap for the cascading approach: color scale is weight (0.0 to 0.4).
    """
    fig, ax = plt.subplots(figsize=(12,6))
    cax = ax.imshow(weight_matrix.T, aspect='auto', interpolation='nearest', origin='lower')
    ax.set_yticks(range(len(sector_labels)))
    ax.set_yticklabels(sector_labels)
    step = max(1, len(dates)//10)
    ax.set_xticks(range(0, len(dates), step))
    xlabels = [dates[i].strftime('%Y-%m-%d') for i in range(0,len(dates), step)]
    ax.set_xticklabels(xlabels, rotation=45, ha='right')
    fig.colorbar(cax, ax=ax, label="Sector Weight")
    ax.set_title("Heatmap: Cascading Weights")
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# -----------------------------------------------------------------------------
# Side-by-side normal frequency bar chart
# -----------------------------------------------------------------------------
def plot_side_by_side_hist(strategy_rets, bench_rets, title, output_path, bins=50):
    """
    Plots a normal frequency histogram of daily returns for both strategy and benchmark
    side-by-side (two bars per bin).
    """
    # Determine a common range for bins based on min/max of both series
    min_val = min(strategy_rets.min(), bench_rets.min())
    max_val = max(strategy_rets.max(), bench_rets.max())

    # Generate bin edges
    bin_edges = np.linspace(min_val, max_val, bins + 1)

    # Compute hist data for each
    hist_strat, _ = np.histogram(strategy_rets, bins=bin_edges)
    hist_bench, _ = np.histogram(bench_rets, bins=bin_edges)

    # Convert bin edges to centers
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    # Width of each bar
    width = (bin_edges[1] - bin_edges[0]) * 0.4  # 40% of the bin width => small gap

    plt.figure(figsize=(8,5))
    # Shift the Strategy bars slightly to the left, Benchmark to the right
    plt.bar(bin_centers - width/2, hist_strat, width=width, label="Strategy")
    plt.bar(bin_centers + width/2, hist_bench, width=width, label="Benchmark")
    plt.xlabel("Daily Returns")
    plt.ylabel("Frequency")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# ---------------------------------
# 6) Main Script
# ---------------------------------
if __name__ == "__main__":
    # =================================================
    # 1) Load data and do once-only transformations
    # =================================================
    df_raw = load_data(FILE_NAME)
    all_price_cols = SECTOR_PRICE_COLS + [BENCHMARK_PRICE_COL]
    df_with_returns = add_daily_return_columns(df_raw, all_price_cols)
    sector_daily_ret_cols = [c + "_daily_ret" for c in SECTOR_PRICE_COLS]
    df_with_fwd = create_180d_forward_returns(df_with_returns, sector_daily_ret_cols, WINDOW)

    # Label columns = "*_daily_ret_fwd180"
    label_cols = [c + "_fwd180" for c in sector_daily_ret_cols]

    # Scale input features
    scaler = MinMaxScaler()
    df_scaled = df_with_fwd.copy()
    df_scaled[INPUT_COLS] = scaler.fit_transform(df_with_fwd[INPUT_COLS])

    # Prepare data for a single LAGS set (the first in LAGS_GRID for example)
    chosen_lags = LAGS_GRID[0]
    X_full, y_full, dates_full = prepare_lagged_data(
        df_scaled, INPUT_COLS, label_cols, chosen_lags
    )

    precomp_train, precomp_eval = split_three_periods_precomputed_data(
        X_full, y_full, dates_full,
        TRAIN1_START, TRAIN1_END,
        VALIDATION_START, VALIDATION_END,
        TRAIN2_START, TRAIN2_END
    )
    X_train, y_train, train_dates = precomp_train
    X_eval, y_eval, eval_dates = precomp_eval

    # Build date-filtered subsets for final predictions
    RANGE_2001_START = pd.to_datetime("2001-01-02")
    RANGE_2001_END   = pd.to_datetime("2011-01-03")
    mask_2001_2011 = [(d >= RANGE_2001_START) and (d <= RANGE_2001_END) for d in dates_full]

    RANGE_2023_START = pd.to_datetime("2021-12-20")
    RANGE_2023_END   = pd.to_datetime("2024-12-18")
    mask_2023_2024 = [(d >= RANGE_2023_START) and (d <= RANGE_2023_END) for d in dates_full]

    X_2001_2011_full = X_full[mask_2001_2011]
    y_2001_2011_full = y_full[mask_2001_2011]
    dates_2001_2011_full = [d for d,m in zip(dates_full,mask_2001_2011) if m]

    X_2023_2024_full = X_full[mask_2023_2024]
    y_2023_2024_full = y_full[mask_2023_2024]
    dates_2023_2024_full = [d for d,m in zip(dates_full,mask_2023_2024) if m]

    # We'll collect a big list of results for the final "grid_search_summary.xlsx"
    all_combinations_results = []   # for the portfolio stats
    all_combinations_hparams = []   # for hyperparameters

    # =================================================
    # 2) Nested loop over hyperparameters
    # =================================================
    for ep in EPOCHS:
        for bs in BATCH_SIZE:
            for dr in DROPOUT:
                for lr in LEARNING_RATE:
                    for reg in REG_LAMBDA:
                        for n_lstm_layers in NUM_LSTM_LAYERS:
                            for first_layer_sz in FIRST_LAYER_SIZE:
                                # Create a subfolder for this combination
                                combo_name = (
                                    f"combination_E{ep}_B{bs}_Dr{dr}_LR{lr}_Reg{reg}_LSTM{n_lstm_layers}"
                                    f"_FS{first_layer_sz}"
                                )
                                combo_output_dir = os.path.join(MAIN_OUTPUT_DIR, combo_name)
                                os.makedirs(combo_output_dir, exist_ok=True)

                                # Also create the various subfolders
                                folder_25_2001_2011 = os.path.join(combo_output_dir, SUBFOLDER_25pct_2001_2011)
                                folder_casc_2001_2011 = os.path.join(combo_output_dir, SUBFOLDER_cascading_2001_2011)
                                folder_25_2023_2024 = os.path.join(combo_output_dir, SUBFOLDER_25pct_2023_2024)
                                folder_casc_2023_2024 = os.path.join(combo_output_dir, SUBFOLDER_cascading_2023_2024)
                                folder_sectors_2001_2011 = os.path.join(combo_output_dir, SUBFOLDER_sectors_2001_2011)
                                folder_sectors_2023_2024 = os.path.join(combo_output_dir, SUBFOLDER_sectors_2023_2024)

                                for path in [
                                    folder_25_2001_2011,
                                    folder_casc_2001_2011,
                                    folder_25_2023_2024,
                                    folder_casc_2023_2024,
                                    folder_sectors_2001_2011,
                                    folder_sectors_2023_2024
                                ]:
                                    os.makedirs(path, exist_ok=True)

                                # ---------------------------------------------------
                                # 2b) Build & Train model
                                # ---------------------------------------------------
                                n_lags = X_train.shape[1]
                                n_features = X_train.shape[2]
                                n_outputs = len(label_cols)
                                model = build_lstm_model(
                                    n_lags=n_lags,
                                    n_features=n_features,
                                    n_outputs=n_outputs,
                                    num_lstm_layers=n_lstm_layers,
                                    first_layer_size=first_layer_sz,
                                    dropout_rate=dr,
                                    reg_lambda=reg,
                                    learning_rate=lr
                                )

                                # Our custom callback for post-15k logic
                                best_model_temp = os.path.join(combo_output_dir, "temp_best_model.keras")
                                post15k_callback = Post15000EarlyStopping(
                                    best_model_path=best_model_temp
                                )

                                # Train
                                history = model.fit(
                                    X_train, y_train,
                                    validation_data=(X_eval, y_eval),
                                    epochs=ep,
                                    batch_size=bs,
                                    verbose=0,
                                    callbacks=[post15k_callback]
                                )

                                # After training, copy the best file to final_lstm_model.keras
                                final_model_path = os.path.join(combo_output_dir, "final_lstm_model.keras")
                                if os.path.exists(best_model_temp):
                                    shutil.copyfile(best_model_temp, final_model_path)
                                else:
                                    # If we never reached epoch 15000 for some reason,
                                    # or if no file was created, just save the final model
                                    model.save(final_model_path)

                                # Plot training vs validation loss
                                plt.figure(figsize=(8,5))
                                plt.plot(history.history['loss'], label='Train MSE Loss')
                                if 'val_loss' in history.history:
                                    plt.plot(history.history['val_loss'], label='Validation MSE Loss')
                                plt.title("Training vs Validation Loss")
                                plt.legend()
                                plt.grid(True)
                                plt.tight_layout()
                                plt.savefig(os.path.join(combo_output_dir, "training_validation_loss.png"))
                                plt.close()

                                # ---------------------------------------------------
                                # 2c) Generate predictions with best model
                                # ---------------------------------------------------
                                loaded_model = load_model(final_model_path)

                                y_pred_2001_2011 = loaded_model.predict(X_2001_2011_full)
                                y_pred_2023_2024 = loaded_model.predict(X_2023_2024_full)

                                # Save sector-level predictions
                                def save_sector_predictions_csv_and_plots(
                                        dates_eval, y_eval, y_pred_eval, label_cols, out_folder
                                ):
                                    os.makedirs(out_folder, exist_ok=True)
                                    df_pred = pd.DataFrame({"Date": dates_eval})
                                    df_pred.set_index("Date", inplace=True)
                                    for idx, col in enumerate(label_cols):
                                        df_pred[f"Actual_{col}"] = y_eval[:, idx]
                                        df_pred[f"Pred_{col}"]   = y_pred_eval[:, idx]

                                        # Quick line plot
                                        plt.figure(figsize=(10,4))
                                        plt.plot(dates_eval, y_eval[:, idx], label="Actual")
                                        plt.plot(dates_eval, y_pred_eval[:, idx], label="Predicted", alpha=0.7)
                                        plt.title(f"{col}: Actual vs Predicted")
                                        plt.legend()
                                        plt.grid(True)
                                        ax = plt.gca()
                                        ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                                        ax.xaxis.set_major_formatter(
                                            mdates.ConciseDateFormatter(ax.xaxis.get_major_locator())
                                        )
                                        plt.tight_layout()
                                        plt.savefig(os.path.join(out_folder, f"actual_vs_pred_{col}.png"))
                                        plt.close()

                                    df_pred.to_csv(os.path.join(out_folder, "sector_predictions.csv"))

                                # 2001–2011 predictions
                                save_sector_predictions_csv_and_plots(
                                    dates_eval=dates_2001_2011_full,
                                    y_eval=y_2001_2011_full,
                                    y_pred_eval=y_pred_2001_2011,
                                    label_cols=label_cols,
                                    out_folder=folder_sectors_2001_2011
                                )

                                # 2023–2024 predictions
                                save_sector_predictions_csv_and_plots(
                                    dates_eval=dates_2023_2024_full,
                                    y_eval=y_2023_2024_full,
                                    y_pred_eval=y_pred_2023_2024,
                                    label_cols=label_cols,
                                    out_folder=folder_sectors_2023_2024
                                )

                                # ---------------------------------------------------
                                # 2d) Run the four portfolios
                                # ---------------------------------------------------
                                df_2001_2011 = df_with_returns.loc[
                                    RANGE_2001_START:RANGE_2001_END,
                                    sector_daily_ret_cols + [BENCHMARK_PRICE_COL + "_daily_ret"]
                                ]
                                df_2023_2024 = df_with_returns.loc[
                                    RANGE_2023_START:RANGE_2023_END,
                                    sector_daily_ret_cols + [BENCHMARK_PRICE_COL + "_daily_ret"]
                                ]

                                # (A1) 25% each, 2001–2011
                                nav_25_2001_2011, top4_25_2001_2011 = backtest_top4_portfolio(
                                    dates_eval=dates_2001_2011_full,
                                    y_pred=y_pred_2001_2011,
                                    df_eval=df_2001_2011,
                                    sector_daily_ret_cols=sector_daily_ret_cols,
                                    benchmark_daily_ret_col=BENCHMARK_PRICE_COL + "_daily_ret",
                                    rebal_freq=REBALANCE_FREQUENCY_DAYS
                                )
                                stats_25_2001_2011 = compute_performance(nav_25_2001_2011["Strategy_NAV"])

                                # Compute daily returns
                                strat_daily_rets_25_2001_2011 = nav_25_2001_2011["Strategy_NAV"].pct_change().dropna()
                                bench_daily_rets_25_2001_2011 = nav_25_2001_2011["Benchmark_NAV"].pct_change().dropna()

                                # Add skew/kurt
                                stats_25_2001_2011["Strategy_Skewness"] = strat_daily_rets_25_2001_2011.skew()
                                stats_25_2001_2011["Strategy_Kurtosis"] = strat_daily_rets_25_2001_2011.kurt()
                                stats_25_2001_2011["Benchmark_Skewness"] = bench_daily_rets_25_2001_2011.skew()
                                stats_25_2001_2011["Benchmark_Kurtosis"] = bench_daily_rets_25_2001_2011.kurt()

                                # NEW: Calculate Beta = Cov(strategy, benchmark) / Var(benchmark)
                                cov = np.cov(strat_daily_rets_25_2001_2011, bench_daily_rets_25_2001_2011)[0,1]
                                var_bench = np.var(bench_daily_rets_25_2001_2011)
                                beta_25_2001_2011 = cov / var_bench if var_bench != 0 else np.nan
                                stats_25_2001_2011["Strategy_Beta"] = beta_25_2001_2011

                                # Plot NAV
                                plt.figure(figsize=(10,5))
                                plt.plot(nav_25_2001_2011.index, nav_25_2001_2011["Strategy_NAV"], label="Strategy")
                                plt.plot(nav_25_2001_2011.index, nav_25_2001_2011["Benchmark_NAV"], label="Benchmark")
                                plt.title("25% Top-4 (2001–2011)")
                                plt.legend()
                                plt.grid(True)
                                ax = plt.gca()
                                ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                                ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
                                plt.tight_layout()
                                plt.savefig(os.path.join(folder_25_2001_2011, "NAV_comparison.png"))
                                plt.close()
                                plot_drawdowns(nav_25_2001_2011,
                                               os.path.join(folder_25_2001_2011, "drawdowns.png"))
                                plot_top4_heatmap(dates_2001_2011_full,
                                                  top4_25_2001_2011,
                                                  sector_daily_ret_cols,
                                                  os.path.join(folder_25_2001_2011, "top4_heatmap.png"))
                                plot_side_by_side_hist(
                                    strat_daily_rets_25_2001_2011,
                                    bench_daily_rets_25_2001_2011,
                                    "Distribution of Daily Returns (25% Top-4 2001–2011)",
                                    os.path.join(folder_25_2001_2011, "hist_strategy_benchmark.png")
                                )

                                # (A2) Cascading, 2001–2011
                                nav_casc_2001_2011, w_casc_2001_2011 = backtest_top4_cascading_weights(
                                    dates_eval=dates_2001_2011_full,
                                    y_pred=y_pred_2001_2011,
                                    df_eval=df_2001_2011,
                                    sector_daily_ret_cols=sector_daily_ret_cols,
                                    benchmark_daily_ret_col=BENCHMARK_PRICE_COL + "_daily_ret",
                                    rebal_freq=REBALANCE_FREQUENCY_DAYS
                                )
                                stats_casc_2001_2011 = compute_performance(nav_casc_2001_2011["Strategy_NAV"])

                                strat_daily_rets_casc_2001_2011 = nav_casc_2001_2011["Strategy_NAV"].pct_change().dropna()
                                bench_daily_rets_casc_2001_2011 = nav_casc_2001_2011["Benchmark_NAV"].pct_change().dropna()

                                stats_casc_2001_2011["Strategy_Skewness"] = strat_daily_rets_casc_2001_2011.skew()
                                stats_casc_2001_2011["Strategy_Kurtosis"] = strat_daily_rets_casc_2001_2011.kurt()
                                stats_casc_2001_2011["Benchmark_Skewness"] = bench_daily_rets_casc_2001_2011.skew()
                                stats_casc_2001_2011["Benchmark_Kurtosis"] = bench_daily_rets_casc_2001_2011.kurt()

                                cov_casc = np.cov(strat_daily_rets_casc_2001_2011, bench_daily_rets_casc_2001_2011)[0,1]
                                var_bench_casc = np.var(bench_daily_rets_casc_2001_2011)
                                beta_casc_2001_2011 = cov_casc / var_bench_casc if var_bench_casc != 0 else np.nan
                                stats_casc_2001_2011["Strategy_Beta"] = beta_casc_2001_2011

                                plt.figure(figsize=(10,5))
                                plt.plot(nav_casc_2001_2011.index, nav_casc_2001_2011["Strategy_NAV"], label="Strategy")
                                plt.plot(nav_casc_2001_2011.index, nav_casc_2001_2011["Benchmark_NAV"], label="Benchmark")
                                plt.title("Cascading Weights Top-4 (2001–2011)")
                                plt.legend()
                                plt.grid(True)
                                ax = plt.gca()
                                ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                                ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
                                plt.tight_layout()
                                plt.savefig(os.path.join(folder_casc_2001_2011, "NAV_comparison.png"))
                                plt.close()
                                plot_drawdowns(nav_casc_2001_2011,
                                               os.path.join(folder_casc_2001_2011, "drawdowns.png"))
                                plot_cascading_weight_heatmap(dates_2001_2011_full,
                                                              w_casc_2001_2011,
                                                              sector_daily_ret_cols,
                                                              os.path.join(folder_casc_2001_2011, "cascading_heatmap.png"))
                                plot_side_by_side_hist(
                                    strat_daily_rets_casc_2001_2011,
                                    bench_daily_rets_casc_2001_2011,
                                    "Distribution of Daily Returns (Cascading Top-4 2001–2011)",
                                    os.path.join(folder_casc_2001_2011, "hist_strategy_benchmark.png")
                                )

                                # (B1) 25% each, 2023–2024
                                nav_25_2023_2024, top4_25_2023_2024 = backtest_top4_portfolio(
                                    dates_eval=dates_2023_2024_full,
                                    y_pred=y_pred_2023_2024,
                                    df_eval=df_2023_2024,
                                    sector_daily_ret_cols=sector_daily_ret_cols,
                                    benchmark_daily_ret_col=BENCHMARK_PRICE_COL + "_daily_ret",
                                    rebal_freq=REBALANCE_FREQUENCY_DAYS
                                )
                                stats_25_2023_2024 = compute_performance(nav_25_2023_2024["Strategy_NAV"])

                                strat_daily_rets_25_2023_2024 = nav_25_2023_2024["Strategy_NAV"].pct_change().dropna()
                                bench_daily_rets_25_2023_2024 = nav_25_2023_2024["Benchmark_NAV"].pct_change().dropna()

                                stats_25_2023_2024["Strategy_Skewness"] = strat_daily_rets_25_2023_2024.skew()
                                stats_25_2023_2024["Strategy_Kurtosis"] = strat_daily_rets_25_2023_2024.kurt()
                                stats_25_2023_2024["Benchmark_Skewness"] = bench_daily_rets_25_2023_2024.skew()
                                stats_25_2023_2024["Benchmark_Kurtosis"] = bench_daily_rets_25_2023_2024.kurt()

                                cov_25_2023_2024 = np.cov(strat_daily_rets_25_2023_2024, bench_daily_rets_25_2023_2024)[0,1]
                                var_bench_25_2023_2024 = np.var(bench_daily_rets_25_2023_2024)
                                beta_25_2023_2024 = cov_25_2023_2024 / var_bench_25_2023_2024 if var_bench_25_2023_2024!=0 else np.nan
                                stats_25_2023_2024["Strategy_Beta"] = beta_25_2023_2024

                                plt.figure(figsize=(10,5))
                                plt.plot(nav_25_2023_2024.index, nav_25_2023_2024["Strategy_NAV"], label="Strategy")
                                plt.plot(nav_25_2023_2024.index, nav_25_2023_2024["Benchmark_NAV"], label="Benchmark")
                                plt.title("25% Top-4 (2023–2024)")
                                plt.legend()
                                plt.grid(True)
                                ax = plt.gca()
                                ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                                ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
                                plt.tight_layout()
                                plt.savefig(os.path.join(folder_25_2023_2024, "NAV_comparison.png"))
                                plt.close()
                                plot_drawdowns(nav_25_2023_2024,
                                               os.path.join(folder_25_2023_2024, "drawdowns.png"))
                                plot_top4_heatmap(dates_2023_2024_full,
                                                  top4_25_2023_2024,
                                                  sector_daily_ret_cols,
                                                  os.path.join(folder_25_2023_2024, "top4_heatmap.png"))
                                plot_side_by_side_hist(
                                    strat_daily_rets_25_2023_2024,
                                    bench_daily_rets_25_2023_2024,
                                    "Distribution of Daily Returns (25% Top-4 2023–2024)",
                                    os.path.join(folder_25_2023_2024, "hist_strategy_benchmark.png")
                                )

                                # (B2) Cascading, 2023–2024
                                nav_casc_2023_2024, w_casc_2023_2024 = backtest_top4_cascading_weights(
                                    dates_eval=dates_2023_2024_full,
                                    y_pred=y_pred_2023_2024,
                                    df_eval=df_2023_2024,
                                    sector_daily_ret_cols=sector_daily_ret_cols,
                                    benchmark_daily_ret_col=BENCHMARK_PRICE_COL + "_daily_ret",
                                    rebal_freq=REBALANCE_FREQUENCY_DAYS
                                )
                                stats_casc_2023_2024 = compute_performance(nav_casc_2023_2024["Strategy_NAV"])

                                strat_daily_rets_casc_2023_2024 = nav_casc_2023_2024["Strategy_NAV"].pct_change().dropna()
                                bench_daily_rets_casc_2023_2024 = nav_casc_2023_2024["Benchmark_NAV"].pct_change().dropna()

                                stats_casc_2023_2024["Strategy_Skewness"] = strat_daily_rets_casc_2023_2024.skew()
                                stats_casc_2023_2024["Strategy_Kurtosis"] = strat_daily_rets_casc_2023_2024.kurt()
                                stats_casc_2023_2024["Benchmark_Skewness"] = bench_daily_rets_casc_2023_2024.skew()
                                stats_casc_2023_2024["Benchmark_Kurtosis"] = bench_daily_rets_casc_2023_2024.kurt()

                                cov_casc_2023_2024 = np.cov(strat_daily_rets_casc_2023_2024, bench_daily_rets_casc_2023_2024)[0,1]
                                var_bench_casc_2023_2024 = np.var(bench_daily_rets_casc_2023_2024)
                                beta_casc_2023_2024 = cov_casc_2023_2024 / var_bench_casc_2023_2024 if var_bench_casc_2023_2024!=0 else np.nan
                                stats_casc_2023_2024["Strategy_Beta"] = beta_casc_2023_2024

                                plt.figure(figsize=(10,5))
                                plt.plot(nav_casc_2023_2024.index, nav_casc_2023_2024["Strategy_NAV"], label="Strategy")
                                plt.plot(nav_casc_2023_2024.index, nav_casc_2023_2024["Benchmark_NAV"], label="Benchmark")
                                plt.title("Cascading Weights Top-4 (2023–2024)")
                                plt.legend()
                                plt.grid(True)
                                ax = plt.gca()
                                ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                                ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
                                plt.tight_layout()
                                plt.savefig(os.path.join(folder_casc_2023_2024, "NAV_comparison.png"))
                                plt.close()
                                plot_drawdowns(nav_casc_2023_2024,
                                               os.path.join(folder_casc_2023_2024, "drawdowns.png"))
                                plot_cascading_weight_heatmap(dates_2023_2024_full,
                                                              w_casc_2023_2024,
                                                              sector_daily_ret_cols,
                                                              os.path.join(folder_casc_2023_2024, "cascading_heatmap.png"))
                                plot_side_by_side_hist(
                                    strat_daily_rets_casc_2023_2024,
                                    bench_daily_rets_casc_2023_2024,
                                    "Distribution of Daily Returns (Cascading Top-4 2023–2024)",
                                    os.path.join(folder_casc_2023_2024, "hist_strategy_benchmark.png")
                                )

                                # ---------------------------------------------------
                                # 2e) Consolidate results into one Excel
                                # ---------------------------------------------------
                                results_data = []
                                results_data.append({"Scenario": "25pct_2001_2011", **stats_25_2001_2011})
                                results_data.append({"Scenario": "cascading_2001_2011", **stats_casc_2001_2011})
                                results_data.append({"Scenario": "25pct_2023_2024", **stats_25_2023_2024})
                                results_data.append({"Scenario": "cascading_2023_2024", **stats_casc_2023_2024})
                                results_df = pd.DataFrame(results_data)

                                # Save hyperparams to a separate sheet
                                hyperparams_data = [
                                    {
                                        "EPOCHS": ep,
                                        "BATCH_SIZE": bs,
                                        "DROPOUT": dr,
                                        "LEARNING_RATE": lr,
                                        "REG_LAMBDA": reg,
                                        "NUM_LSTM_LAYERS": n_lstm_layers,
                                        "FIRST_LAYER_SIZE": first_layer_sz
                                    }
                                ]
                                hyperparams_df = pd.DataFrame(hyperparams_data)

                                excel_path = os.path.join(combo_output_dir, "all_portfolios_results.xlsx")
                                with pd.ExcelWriter(excel_path) as writer:
                                    results_df.to_excel(writer, sheet_name="Portfolio_Results", index=False)
                                    hyperparams_df.to_excel(writer, sheet_name="Hyperparameters", index=False)

                                # We'll also store these results in a global list
                                for row in results_data:
                                    row["EPOCHS"] = ep
                                    row["BATCH_SIZE"] = bs
                                    row["DROPOUT"] = dr
                                    row["LEARNING_RATE"] = lr
                                    row["REG_LAMBDA"] = reg
                                    row["NUM_LSTM_LAYERS"] = n_lstm_layers
                                    row["FIRST_LAYER_SIZE"] = first_layer_sz
                                    all_combinations_results.append(row)

                                # Also store a single row (with no scenario) in the hyperparams summary
                                all_combinations_hparams.append({
                                    "EPOCHS": ep,
                                    "BATCH_SIZE": bs,
                                    "DROPOUT": dr,
                                    "LEARNING_RATE": lr,
                                    "REG_LAMBDA": reg,
                                    "NUM_LSTM_LAYERS": n_lstm_layers,
                                    "FIRST_LAYER_SIZE": first_layer_sz
                                })

                                # ---------------------------------------------------
                                # 2f) (Optional) Zip up just this combination
                                # ---------------------------------------------------
                                zip_path_combo = os.path.join(combo_output_dir, f"{combo_name}.zip")
                                with zipfile.ZipFile(zip_path_combo, 'w', zipfile.ZIP_DEFLATED) as zipf:
                                    for root, dirs, files in os.walk(combo_output_dir):
                                        # Don't recursively add the zip inside itself
                                        for file in files:
                                            if file == f"{combo_name}.zip":
                                                continue
                                            file_path = os.path.join(root, file)
                                            arcname = os.path.relpath(file_path, start=combo_output_dir)
                                            zipf.write(file_path, arcname)

    # ================================================================
    # 3) After finishing ALL combos, create a big summary in MAIN_OUTPUT_DIR
    # ================================================================
    master_summary_path = os.path.join(MAIN_OUTPUT_DIR, "grid_search_summary.xlsx")

    all_results_df = pd.DataFrame(all_combinations_results)
    all_hparams_df = pd.DataFrame(all_combinations_hparams).drop_duplicates()

    with pd.ExcelWriter(master_summary_path) as writer:
        all_results_df.to_excel(writer, sheet_name="All_Combos_Portfolio_Results", index=False)
        all_hparams_df.to_excel(writer, sheet_name="All_Combos_Hyperparameters", index=False)

    # ================================================================
    # 4) (Optional) Zip the entire MAIN_OUTPUT_DIR if desired
    # ================================================================
    zip_path = "grid_search_results_LSTM.zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(MAIN_OUTPUT_DIR):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=MAIN_OUTPUT_DIR)
                zipf.write(file_path, arcname)
    print(f"All results have been zipped into: {zip_path}")

    # Attempt to download if in Colab (optional)
    try:
        from google.colab import files
        files.download(zip_path)
    except ImportError:
        pass

    print("All hyperparameter combinations are complete.")
    print(f"Master summary saved to: {master_summary_path}")
    print(f"Zipped folder created at: {zip_path}")
    print("Script complete.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Early stopping at epoch 27958 because val_loss (0.050579) exceeded 105% of best_val_loss (0.047528).
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
All results have been zipped into: grid_search_results_LSTM.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All hyperparameter combinations are complete.
Master summary saved to: grid_search_results_LSTM/grid_search_summary.xlsx
Zipped folder created at: grid_search_results_LSTM.zip
Script complete.


In [ ]:
    zip_path = "grid_search_results_LSTM.zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(MAIN_OUTPUT_DIR):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=MAIN_OUTPUT_DIR)
                zipf.write(file_path, arcname)
    print(f"All results have been zipped into: {zip_path}")

    # Attempt to download if in Colab (optional)
    try:
        from google.colab import files
        files.download(zip_path)
    except ImportError:
        pass

All results have been zipped into: grid_search_results_LSTM.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>